<a href="https://colab.research.google.com/github/Frankossin/Trabalho-Franklin/blob/Modelagem-fisica/Formas_normais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Guilherme Schwarz
Franklin Jeronimo
Caua Gregorio
Julia Moreira

Nota: Percebemos que as vezes o arquivo de texto data.txt é deletado do Collab

"""

from IPython.display import display, Latex

def lastIndex(arr: list, element):
    return len(arr) - arr[::-1].index(element) - 1

def wrongParentesis(arr: list):
    if arr.count("(") + arr.count(")") == 0:
        return False

    if arr.count("(") != arr.count(")"):
        return True

    for i in range( len(arr) - 1 ):
        if arr[i] == "(" and arr[i + 1] == ")":
            return True
        if arr[i] == ")" and arr[i + 1] == "(":
            return True

    if arr.index(")") < arr.index("("):
        return True

    if lastIndex(arr, ")") < lastIndex(arr, "("):
        return True

    return False

def wrongOperators(arr: list):
    if isOperator(arr[0]) and isOperator(arr[len(arr) - 1]):
        return True

    for i in range( len(arr) - 1 ):
        if isOperator(arr[i]):
            if isOperator(arr[i + 1]):
                return True

            if arr[i + 1] == "(":
                if isOperator(arr[i + 2]):
                    return True

            if isOperator(arr[i + 1]) == "\\neg":
                return True

    return False

def valid(arr: list):
    if wrongParentesis(arr):
        return False

    if wrongOperators(arr):
        return False

    return True

def findStartEnd(arr, opIndex):
    start = opIndex - 1 if arr[opIndex - 2] != "\\neg" else opIndex - 2
    end = opIndex + 2 if arr[opIndex + 1] != "\\neg" else opIndex + 3

    level = 0
    if arr[opIndex - 1] == ")":
        for i in range(opIndex - 2, -1, -1):
            if arr[i] == ")":
                level += 1

            if arr[i] == "(":
                if level == 0:
                    start = i
                    break
                else:
                    level -= 1
    if arr[opIndex + 1] == "(":
        for i in range(opIndex + 2, len(arr)):
            if arr[i] == "(":
                level += 1

            if arr[i] == ")":
                if level == 0:
                    end = i + 1
                    break
                else:
                    level -= 1

    return start, end

def transBicon(a, b):
    a = a if isinstance(a, list) else [a]
    b = b if isinstance(b, list) else [b]

    return ["("] + a + ["\\rightarrow"] + b + [")", "\\wedge", "("] + b + ["\\rightarrow"] + a + [")"]

def transImp(a, b):
    a = a if isinstance(a, list) else [a]
    b = b if isinstance(b, list) else [b]

    return ["\\neg"] + a + ["\\vee"] + b

def solveBicon(arr: list):
    opIndex = arr.index("\\leftrightarrow")

    start, end = findStartEnd(arr, opIndex)

    a = arr[start : opIndex]
    b = arr[opIndex + 1 : end]
    del arr[start: end]

    result = transBicon(a, b)
    arr = arr[0: start] + result + arr[start: len(arr)]
    return arr

def solveImp(arr: list):
    opIndex = arr.index("\\rightarrow")

    start, end = findStartEnd(arr, opIndex)

    a = arr[start : opIndex]
    b = arr[opIndex + 1 : end]
    del arr[start: end]

    arr = arr[0: start] + transImp(a, b) + arr[start: len(arr)]
    return arr

def getChild(arr: list):
    i = 0
    children = []
    while i < len(arr):
        if arr[i] != "(":
                children.append([arr[i]])
                i += 1
                continue
        level = 0
        for u in range(i + 1, len(arr)):
            if arr[u] == "(":
                level += 1
            elif arr[u] == ")":
                if level == 0:
                    children.append(arr[i:u + 1])
                    i = u + 1
                    break
                else:
                    level -= 1
    return children

def reverse(operator):
    if operator == "\\wedge":
        return "\\vee"
    else:
        return "\\wedge"

def applyNeg(children: list):
    result = []
    for child in children:
        if child[0] in ["\\wedge", "\\vee"]:
            result += [reverse(child[0])]
        elif child[0] == "\\neg":
            result += ["\\neg"]
        else:
            result += ["\\neg"] + child
    return result

def solveNeg(arr: list):
    for i in range(len(arr) - 1):
        if arr[i] == "\\neg" and arr[i + 1] == "(":
            start = i + 1
            break
    level = 0
    for i in range(start + 1, len(arr)):
        if arr[i] == "(":
            level += 1

        elif arr[i] == ")":
            if level == 0:
                end = i + 1
                break
            else:
                level -= 1

    children = getChild(arr[start + 1: end - 1])
    del arr[start - 1: end]

    arr = arr[0: start - 1] + ["("] + applyNeg(children) + [")"] + arr[start - 1: len(arr)]
    return arr

def removeNeg(arr: list):
    for i in range(len(arr) - 1):
        if arr[i] == "\\neg" and arr[i + 1] == "\\neg":
            return arr[0: i] + arr[i + 2: len(arr)]
    return arr

def isOperator(var):
    if var in ["\\leftrightarrow", "\\rightarrow", "\\wedge", "\\vee"]:
        return True
    return False

def canDistribute(arr: list):
    for i in range( len(arr) - 1 ):
        if arr[i] == "\\vee" and arr[i + 1] == "(":
            u = i + 2
            level = 0
            while u < len(arr):
                if arr[u] == "(":
                    level += 1
                elif arr[u] == ")":
                    level -= 1
                    if level < 0:
                        break
                elif arr[u] == "\\wedge":
                    if level == 0:
                        return True
                u += 1
    return False

def canDistribute_wedge(arr: list):
    for i in range( len(arr) - 1 ):
        if arr[i] == "\\wedge" and arr[i + 1] == "(":
            u = i + 2
            level = 0
            while u < len(arr):
                if arr[u] == "(":
                    level += 1
                elif arr[u] == ")":
                    level -= 1
                    if level < 0:
                        break
                elif arr[u] == "\\vee":
                    if level == 0:
                        return True
                u += 1
    return False

def transDistribute(a,b,c):
    a = a if isinstance(a, list) else [a]
    b = b if isinstance(b, list) else [b]
    c = c if isinstance(c, list) else [c]

    return ["("] + a + ["\\vee"] + b + [")"] + ["\\wedge"] + ["("] + a + ["\\vee"] + c + [")"]

def transDistribute_wedge(a,b,c):
    a = a if isinstance(a, list) else [a]
    b = b if isinstance(b, list) else [b]
    c = c if isinstance(c, list) else [c]

    return ["("] + a + ["\\wedge"] + b + [")"] + ["\\vee"] + ["("] + a + ["\\wedge"] + c + [")"]

def distribute(arr:list):
    for i in range( len(arr) - 1 ):
        if arr[i] == "\\vee" and arr[i + 1] == "(":
            u = i + 2
            level = 0
            while u < len(arr):
                if arr[u] == "(":
                    level += 1
                elif arr[u] == ")":
                    level -= 1
                    if level < 0:
                        break
                elif arr[u] == "\\wedge":
                    if level == 0:
                        A, P = findStartEnd(arr, i)
                        B, C = findStartEnd(arr, u)

                        a = arr[A: B - 2]
                        b = arr[B: u]
                        c = arr[u + 1: C ]
                        del arr[A: P]

                        result = transDistribute(a, b, c)
                        arr = arr[0: A] + result + arr[A: len(arr)]

                        return arr
                u += 1

def distribute_wedge(arr:list):
    for i in range( len(arr) - 1 ):
        if arr[i] == "\\wedge" and arr[i + 1] == "(":
            u = i + 2
            level = 0
            while u < len(arr):
                if arr[u] == "(":
                    level += 1
                elif arr[u] == ")":
                    level -= 1
                    if level < 0:
                        break
                elif arr[u] == "\\vee":
                    if level == 0:
                        A, P = findStartEnd(arr, i)
                        B, C = findStartEnd(arr, u)

                        a = arr[A: B-2]
                        b = arr[B: u]
                        c = arr[u + 1: C ]
                        del arr[A: P]

                        result = transDistribute_wedge(a,b,c)
                        arr = arr[0: A] + result + arr[A: len(arr)]

                        return arr
                u += 1

def cnf(arr: list):
    if "\\leftrightarrow" in arr:
        arr = solveBicon(arr)
        return cnf(arr)
    if "\\rightarrow" in arr:
        arr = solveImp(arr)
        return cnf(arr)
    if "\\neg" in arr:
        for i in range(len(arr) - 1):
            if arr[i] == "\\neg" and arr[i + 1] == "(":
                arr = solveNeg(arr)
                return cnf(arr)
        for i in range(len(arr) - 1):
            if arr[i] == "\\neg" and arr[i + 1] == "\\neg":
                arr = removeNeg(arr)
                return cnf(arr)
    if canDistribute(arr):
        arr = distribute(arr)
        return cnf(arr)
    return arr

def dnf(arr: list):
    if "\\leftrightarrow" in arr:
        arr = solveBicon(arr)
        return dnf(arr)
    if "\\rightarrow" in arr:
        arr = solveImp(arr)
        return dnf(arr)
    if "\\neg" in arr:
        for i in range(len(arr) - 1):
            if arr[i] == "\\neg" and arr[i + 1] == "(":
                arr = solveNeg(arr)
                return dnf(arr)
        for i in range(len(arr) - 1):
            if arr[i] == "\\neg" and arr[i + 1] == "\\neg":
                arr = removeNeg(arr)
                return dnf(arr)
    if canDistribute_wedge(arr):
        arr = distribute_wedge(arr)
        return dnf(arr)
    return arr

with open("data.txt") as f:
    funcs = f.read().splitlines()

for func in funcs:
    arr = func.replace("\n", "").split(" ")
    if valid( arr ):
        arr2 = arr.copy()
        print("\nForma inicial: ")
        display(Latex(" ".join(arr)))
        print("Forma normal conjuntiva")
        display(Latex(" ".join(cnf( arr ))))
        print("Forma normal dijuntiva")
        display(Latex(" ".join(dnf( arr2 ))))
    else:
        print("A formula dada não é valida")




Forma inicial: 


<IPython.core.display.Latex object>

Forma normal conjuntiva


<IPython.core.display.Latex object>

Forma normal dijuntiva


...

<IPython.core.display.Latex object>

Forma normal conjuntiva


<IPython.core.display.Latex object>

Forma normal dijuntiva


<IPython.core.display.Latex object>